Alterar a gpu para GPU em tempo de execução

Instalar as dependencias

In [1]:
!pip install -U pip
!pip install -U transformers datasets peft accelerate trl safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 72.8 MB/s  0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [trl]


Montar o Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Criar a pasta do projeto

In [3]:
import os
PROJECT_DIR = "/content/drive/MyDrive/medical-assistant-colab"
os.makedirs(PROJECT_DIR, exist_ok=True)

Faça upload manual de:

*   train.jsonl
*   valid.jsonl





In [5]:
from google.colab import files
uploaded = files.upload()
print(uploaded.keys())

Saving train.jsonl to train.jsonl
Saving valid.jsonl to valid.jsonl
dict_keys(['train.jsonl', 'valid.jsonl'])


Mover os arquivos

In [6]:
import shutil

for fn in uploaded.keys():
    shutil.move(fn, os.path.join(PROJECT_DIR, fn))

!ls -lah "{PROJECT_DIR}"


total 185K
drwx------ 2 root root 4.0K Jan  2 21:01 outputs
-rw------- 1 root root 163K Jan  2 22:22 train.jsonl
-rw------- 1 root root  18K Jan  2 22:22 valid.jsonl


Carregar o dataset

In [7]:
from datasets import load_dataset
import os

dataset = load_dataset(
    "json",
    data_files={
        "train": os.path.join(PROJECT_DIR, "train.jsonl"),
        "validation": os.path.join(PROJECT_DIR, "valid.jsonl")
    }
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Formatar os exemplos para treino

In [8]:
def format_example(ex):
    return {
        "text": (
            "### Instrução:\n" + ex["instruction"] + "\n\n"
            "### Entrada:\n" + ex["input"] + "\n\n"
            "### Resposta:\n" + ex["output"]
        )
    }

dataset2 = dataset.map(
    format_example,
    remove_columns=dataset["train"].column_names
)

print(dataset2["train"][0]["text"][:400])


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

### Instrução:
Gere um rascunho de laudo clínico estruturado.

### Entrada:
Dados do paciente: {'idade': 18, 'sexo': 'F', 'satO2': 93, 'temperatura': 37.5, 'sintomas': 'dispneia leve e febre', 'comorbidades': 'hipertensão'}

### Resposta:
LAUDO (RASCUNHO)
Queixa: ...
Achados clínicos: ...
Hipóteses diagnósticas: ...
Exames sugeridos: ...
Alertas: ...
Plano: encaminhar para avaliação médica.


Carregar modelo e treinar - Carregar modelo base (sem quantização)

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Configurar LoRA

In [10]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


Configurar o treinamento (SEM AMP)

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=os.path.join(PROJECT_DIR, "outputs"),
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=20,
    save_steps=200,
    save_total_limit=2,
    fp16=False,
    bf16=False,
    report_to="none"
)


Criar o Trainer (modo compatível)

In [12]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset2["train"],
    peft_config=lora_config,
    args=training_args
)


Adding EOS to train dataset:   0%|          | 0/360 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/360 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/360 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Treinar o modelo

In [13]:
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
20,1.857100
40,0.987900


TrainOutput(global_step=45, training_loss=1.3385937637752956, metrics={'train_runtime': 96.2282, 'train_samples_per_second': 3.741, 'train_steps_per_second': 0.468, 'total_flos': 844090639687680.0, 'train_loss': 1.3385937637752956, 'entropy': 0.9095703125, 'num_tokens': 50633.0, 'mean_token_accuracy': 0.8545518830418587, 'epoch': 1.0})

Salvar e baixar o resultado
Salvar o adapter LoRA

In [14]:
adapter_dir = os.path.join(PROJECT_DIR, "lora_adapter")
trainer.model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)


('/content/drive/MyDrive/medical-assistant-colab/lora_adapter/tokenizer_config.json',
 '/content/drive/MyDrive/medical-assistant-colab/lora_adapter/special_tokens_map.json',
 '/content/drive/MyDrive/medical-assistant-colab/lora_adapter/chat_template.jinja',
 '/content/drive/MyDrive/medical-assistant-colab/lora_adapter/vocab.json',
 '/content/drive/MyDrive/medical-assistant-colab/lora_adapter/merges.txt',
 '/content/drive/MyDrive/medical-assistant-colab/lora_adapter/added_tokens.json',
 '/content/drive/MyDrive/medical-assistant-colab/lora_adapter/tokenizer.json')

Zipar e baixar

In [15]:
import shutil
zip_path = shutil.make_archive(
    os.path.join(PROJECT_DIR, "lora_adapter"),
    "zip",
    adapter_dir
)


Baixar

In [16]:
from google.colab import files
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>